In [1]:
import glob
from osgeo import gdal, osr

In [2]:
import os
import numpy as np
import dask.array as da
import time


In [3]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return sorted(FilesList),sorted(FileName)
        else:
            print("no"+ extension +"file for this directory")

In [7]:
folder_list=glob.glob('../geoserver_data/2020_NDVI/*/')
folder_list[0]

'../geoserver_data/2020_Cop_LAI/2000/'

In [4]:
folder_list2=['../data/cop_ndvi/2005/','../data/cop_ndvi/2019/']

In [ ]:
# You should give list of nc files' path like folder_list2 or folder_list

In [ ]:
%%time
for folder in folder_list2:
    image_status=False
    counter=0
    while image_status is False:
        try:
            nc_files=ListofExtensionAndName(folder,'.nc')
            folder_name=folder.split('/')[-2]
            output_folder='../data/cop_ndvi/mean_tif/'
            array_list=[]
            for index,img in enumerate(nc_files[0]):
                # with this block, we read 36 .nc file' ndvi value
                # with dask array
                netcdf_name=img
                # if you work with LAI change the variable
                #layer_name='LAI'
                layer_name='NDVI'
                ds = gdal.Open("NETCDF:{0}:{1}".format(netcdf_name, layer_name))
                array=ds.ReadAsArray()
                dask_array=da.from_array(array,chunks=len(array) // 4)
                array_list.append(dask_array)
             
            
            stack = da.stack(array_list, axis=0)
            mean_array=stack.mean(axis=0).compute()
            s_srs = ds.GetProjectionRef()    
            osng = osr.SpatialReference ()
            
            osng.SetFromUserInput ( s_srs )    
            geo_t = ds.GetGeoTransform ()    
            x_size = ds.RasterXSize # Raster xsize
            y_size = ds.RasterYSize # Raster ysize
            mem_drv = gdal.GetDriverByName( 'MEM')
            dest = mem_drv.Create('', x_size,y_size, 1)
            dest.SetGeoTransform( geo_t )
            dest.SetProjection( osng.ExportToWkt())
            dest.GetRasterBand(1).WriteArray(mean_array)
            output_mean_path=output_folder+'c_gls_NDVI_Mean_'+folder_name+'.tif'            
            gdal.Warp(output_mean_path, dest, format = 'GTiff')
            print(output_mean_path)
            mean_array=None
            dest=None
            ds=None
            mem_drv=None
            image_status=True
        except:
            print('except')
            image_status=False
            counter+=1
            if counter==10:
                print(folder + 'couldnt be calculated')
                break
            continue